# Review data preparation

In [85]:
%load_ext autoreload
%autoreload 2

# Import ds libraries
import pandas as pd
import numpy as np
import re

from datetime import datetime, timedelta

# Import acquire functions
import nick_acquire as a
import nick_prepare as prep

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
api_reviews = pd.read_csv('reviews_progress.csv')
scrape_reviews = pd.read_csv('scrapted_reviews.csv')

In [27]:
scrape_reviews.head(3)

,id,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,url_user,r_additional
0,41375676,ChZDSUhNMG9nS0VJQ0FnSUNGMDhHSkVnEAE,NaN,11 hours ago,2023-11-02 02:44:04.226505,5.0,Eddie Trujillo,1,https://www.google.com/maps/contrib/1064176602...,"['Service: 5', 'Atmosphere: 5']"
1,41375676,ChdDSUhNMG9nS0VJQ0FnSUNGM2NhcGpnRRAB,NaN,a day ago,2023-11-02 02:44:04.227451,5.0,Mahamadou Asouman,0,https://www.google.com/maps/contrib/1078602531...,"['Service: 5', 'Atmosphere: 5']"
2,41375676,ChdDSUhNMG9nS0VJQ0FnSUNGbmVidnRRRRAB,NaN,a day ago,2023-11-02 02:44:04.227954,5.0,sedjnie ritoe,0,https://www.google.com/maps/contrib/1170344005...,[]


In [34]:
scrape_reviews.relative_date = scrape_reviews.relative_date.apply(lambda x: x[:-4])
scrape_reviews.relative_date = ['1 years' if date == 'a year' else date for date in scrape_reviews.relative_date]
scrape_reviews.head(3)

,id,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,url_user,r_additional
0,41375676,ChZDSUhNMG9nS0VJQ0FnSUNGMDhHSkVnEAE,NaN,11 hours,2023-11-02 02:44:04.226505,5.0,Eddie Trujillo,1,https://www.google.com/maps/contrib/1064176602...,"['Service: 5', 'Atmosphere: 5']"
1,41375676,ChdDSUhNMG9nS0VJQ0FnSUNGM2NhcGpnRRAB,NaN,a day,2023-11-02 02:44:04.227451,5.0,Mahamadou Asouman,0,https://www.google.com/maps/contrib/1078602531...,"['Service: 5', 'Atmosphere: 5']"
2,41375676,ChdDSUhNMG9nS0VJQ0FnSUNGbmVidnRRRRAB,NaN,a day,2023-11-02 02:44:04.227954,5.0,sedjnie ritoe,0,https://www.google.com/maps/contrib/1170344005...,[]


In [10]:
scrape_reviews.id.unique()

array([41375676, 50093783, 50072407, 40401445, 50072544, 41585586,
       50105651, 50014375, 50110343, 41425365, 50035490, 50019166,
       50060865, 41681629, 50041581, 50005481, 50018654, 50085183,
       50033274, 50044965, 50127886])

In [80]:
p1 = scrape_reviews[scrape_reviews.id == 50093783]
review_counts = pd.DataFrame(p1.relative_date.value_counts())
review_counts['increment'] = 365/review_counts.relative_date
review_counts[:10]

,relative_date,increment
4 years,62,5.887097
3 years,34,10.735294
1 years,30,12.166667
2 years,20,18.250000
5 years,16,22.812500
10 months,4,91.250000
2 months,4,91.250000
11 months,3,121.666667
8 months,3,121.666667
6 months,3,121.666667


In [50]:
review_counts.loc['4 years'].increment

5.887096774193548

In [67]:
restaurant = scrape_reviews[scrape_reviews.id == 50093783].copy()

place = scrape_reviews[scrape_reviews.id == 50093783]
review_counts = pd.DataFrame(place.relative_date.value_counts())
review_counts['increment'] = 365/review_counts.relative_date

new_dates = []
i = 0
previous_year = '1 years'
for date in restaurant.relative_date:  # For every date
    if 'years' in date:
        if date != previous_year:
            i = 0
            previous_year = date
        adjusted_date = (365*(int(date[0]))) + (review_counts.loc[date].increment * i)
        i += 1
        new_dates.append(adjusted_date)
    else:
        new_dates.append(date)
restaurant['new_date'] = new_dates
restaurant

,id,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,url_user,r_additional,new_date
200,50093783,ChdDSUhNMG9nS0VJQ0FnSUNGcGY3VndRRRAB,The plantains were made on the spot in a rush ...,3 days,2023-11-02 02:47:30.895865,3.0,Six Seven Entertainment,0,https://www.google.com/maps/contrib/1068951192...,"['Service Take out', 'Price per person $10–20'...",3 days
201,50093783,ChdDSUhNMG9nS0VJQ0FnSUQtaWN1STBBRRAB,"It was my first time at this restaurant, and w...",2 weeks,2023-11-02 02:47:30.896429,1.0,S W,0,https://www.google.com/maps/contrib/1143606720...,[],2 weeks
202,50093783,ChZDSUhNMG9nS0VJQ0FnSUNaNWJpaUZBEAE,"I went there to get breakfast, at exactly 8am ...",a month,2023-11-02 02:47:30.897581,1.0,Delicia Singh,0,https://www.google.com/maps/contrib/1064364911...,"['Service Take out', 'Meal type Breakfast', 'P...",a month
203,50093783,ChZDSUhNMG9nS0VJQ0FnSUNac3JpM013EAE,Ordered online thru door dash and they sent th...,a month,2023-11-02 02:47:30.898268,1.0,Ashley White,0,https://www.google.com/maps/contrib/1160471953...,"['Food: 1', 'Service: 1', 'Atmosphere: 1']",a month
204,50093783,ChdDSUhNMG9nS0VJQ0FnSURwNTVhU3NBRRAB,NaN,2 months,2023-11-02 02:47:30.899326,3.0,Bellman Rodriguez Quiroga,63,https://www.google.com/maps/contrib/1147885710...,"['Meal type Lunch', 'Price per person $10–20',...",2 months
...,...,...,...,...,...,...,...,...,...,...,...
386,50093783,ChZDSUhNMG9nS0VJQ0FnSUNBejUzd0pnEAE,I bought three chickens and the chickens were ...,5 years,2023-11-02 02:49:53.843048,1.0,Mr.García,6,https://www.google.com/maps/contrib/1065367642...,[],2075.9375
387,50093783,ChdDSUhNMG9nS0VJQ0FnSUN3b3RUdTdRRRAB,From what I'm told the food supposed to be pr...,5 years,2023-11-02 02:49:53.843549,1.0,k foote,0,https://www.google.com/maps/contrib/1101870184...,[],2098.75
388,50093783,ChZDSUhNMG9nS0VJQ0FnSUNRcGEyMFFBEAE,The best Latin food in the area.,5 years,2023-11-02 02:49:53.844049,5.0,miguel gil,0,https://www.google.com/maps/contrib/1072434923...,[],2121.5625
389,50093783,ChZDSUhNMG9nS0VJQ0FnSUN3a1pxNUVBEAE,The Food Was Amazing. Everything had a fresh t...,5 years,2023-11-02 02:49:53.844547,5.0,Nick Paradise,4,https://www.google.com/maps/contrib/1164768180...,[],2144.375


In [93]:
dataframes = []
for restaurant_id in scrape_reviews.id.unique():  # For each restaurant id
    restaurant = scrape_reviews[scrape_reviews.id == restaurant_id].copy()

    place = scrape_reviews[scrape_reviews.id == restaurant_id]
    review_counts = pd.DataFrame(place.relative_date.value_counts())
    review_counts['increment'] = 365/review_counts.relative_date

    new_dates = []
    i = 0
    previous_year = '1 years'
    
    for date in restaurant.relative_date:  # For every date
        if 'years' in date:
            if date != previous_year:
                i = 0
                previous_year = date
            adjusted_date = (365*(int(date[0]))) + (review_counts.loc[date].increment * i)
            i += 1
            new_dates.append(str(round(adjusted_date)))
        else:
            new_dates.append(date)
    restaurant['new_date'] = new_dates
    dataframes.append(restaurant)
    

In [94]:
reviews = pd.concat(dataframes)

In [99]:
reviews.new_date = [re.sub(r'^a', '1', date) if date[0] == 'a' else date for date in reviews.new_date]

In [103]:
new_date = []
for date in reviews.new_date:
    #print(date)
    unit = re.sub(r'[^a-z]', '', date)
    if 'hour' in unit:
        new_date.append('1')
    elif 'day' in unit:
        new_date.append(re.sub(r'[^0-9]', '', date))
    elif 'week' in unit:
        new_date.append(int(re.sub(r'[^0-9]', '', date))*7)
    elif 'month' in unit:
        new_date.append(int(re.sub(r'[^0-9]', '', date))*30)
    else:
        new_date.append(date)
        
reviews['newer_dates'] = new_date

In [120]:
reviews['final_date'] = [pd.to_datetime(retrieval_date) - timedelta(days = n) for retrieval_date,n in zip(reviews.retrieval_date, reviews.newer_dates.astype(int))]

In [121]:
reviews

,id,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,url_user,r_additional,new_date,newer_dates,final_date
0,41375676,ChZDSUhNMG9nS0VJQ0FnSUNGMDhHSkVnEAE,NaN,11 hours,2023-11-02 02:44:04.226505,5.0,Eddie Trujillo,1,https://www.google.com/maps/contrib/1064176602...,"['Service: 5', 'Atmosphere: 5']",11 hours,1,2023-11-01 02:44:04.226505
1,41375676,ChdDSUhNMG9nS0VJQ0FnSUNGM2NhcGpnRRAB,NaN,a day,2023-11-02 02:44:04.227451,5.0,Mahamadou Asouman,0,https://www.google.com/maps/contrib/1078602531...,"['Service: 5', 'Atmosphere: 5']",1 day,1,2023-11-01 02:44:04.227451
2,41375676,ChdDSUhNMG9nS0VJQ0FnSUNGbmVidnRRRRAB,NaN,a day,2023-11-02 02:44:04.227954,5.0,sedjnie ritoe,0,https://www.google.com/maps/contrib/1170344005...,[],1 day,1,2023-11-01 02:44:04.227954
3,41375676,ChZDSUhNMG9nS0VJQ0FnSUNGNmY2Q0F3EAE,Fabulous food,4 days,2023-11-02 02:44:04.228882,5.0,Vibhuti Gedela,20,https://www.google.com/maps/contrib/1161880486...,"['Service Dine in', 'Meal type Dinner', 'Price...",4 days,4,2023-10-29 02:44:04.228882
4,41375676,ChZDSUhNMG9nS0VJQ0FnSUNGNmZEdEVnEAE,"Overrated. Terrible service, the fault of the...",4 days,2023-11-02 02:44:04.229576,3.0,Giulia Passaniti,107,https://www.google.com/maps/contrib/1061104226...,[],4 days,4,2023-10-29 02:44:04.229576
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3390,50127886,ChdDSUhNMG9nS0VJQ0FnSURtbExxdTJRRRAB,NaN,1 years,2023-11-02 03:48:08.598688,4.0,Shiying Zheng,21,https://www.google.com/maps/contrib/1136731733...,"['Meal type Brunch', 'Price per person $20–30'...",705,705,2021-11-27 03:48:08.598688
3391,50127886,ChZDSUhNMG9nS0VJQ0FnSURtaUlDWUlREAE,NaN,1 years,2023-11-02 03:48:08.599530,5.0,Y Y (AllDayLong),6,https://www.google.com/maps/contrib/1022080471...,"['Meal type Lunch', 'Price per person $20–30']",710,710,2021-11-22 03:48:08.599530
3392,50127886,ChZDSUhNMG9nS0VJQ0FnSURtZ0xmbGJnEAE,The vermicelli is a bit thick and the taste is...,1 years,2023-11-02 03:48:08.600083,4.0,yifang wu,132,https://www.google.com/maps/contrib/1107417919...,[],715,715,2021-11-17 03:48:08.600083
3393,50127886,ChZDSUhNMG9nS0VJQ0FnSUNtbjhxTkFREAE,Literally the best egg custard bun I’ve ever had.,1 years,2023-11-02 03:48:08.600817,5.0,Stephanie Heath,21,https://www.google.com/maps/contrib/1165764827...,"['Service Delivery', 'Meal type Dinner', 'Pric...",720,720,2021-11-12 03:48:08.600817
